<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML311Coursera747-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Machine Learning Foundation

## Course 5, Part g: RNN DEMO


## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).


In [1]:
%%capture
!pip install --upgrade tensorflow

In [5]:
from tensorflow import keras
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [6]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [7]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [8]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [9]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [12]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [13]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 50)          1000000   
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 5)                 280       
                                                                 
 dense_1 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [14]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [15]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 9s 9ms/step - loss: 0.6694 - accuracy: 0.6009 - val_loss: 0.6185 - val_accuracy: 0.6671
Epoch 2/10
782/782 [==============================] - 7s 9ms/step - loss: 0.5702 - accuracy: 0.7068 - val_loss: 0.5502 - val_accuracy: 0.7137
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.5043 - accuracy: 0.7565 - val_loss: 0.5114 - val_accuracy: 0.7424
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4618 - accuracy: 0.7842 - val_loss: 0.4882 - val_accuracy: 0.7580
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.4313 - accuracy: 0.8003 - val_loss: 0.4747 - val_accuracy: 0.7676
Epoch 6/10
782/782 [==============================] - 8s 11ms/step - loss: 0.4093 - accuracy: 0.8130 - val_loss: 0.4626 - val_accuracy: 0.7744
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3935 - accuracy: 0.8223 - val_loss: 0.4571 - val_accuracy: 0.7785
Ep

In [16]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 2s 3ms/step - loss: 0.4524 - accuracy: 0.7849
Test score: 0.45244479179382324
Test accuracy: 0.7849199771881104


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?


In [17]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [18]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [19]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [20]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 14s 16ms/step - loss: 0.6627 - accuracy: 0.5757 - val_loss: 0.6186 - val_accuracy: 0.7195
Epoch 2/10
782/782 [==============================] - 11s 15ms/step - loss: 0.5868 - accuracy: 0.7492 - val_loss: 0.5911 - val_accuracy: 0.7414
Epoch 3/10
782/782 [==============================] - 12s 15ms/step - loss: 0.5356 - accuracy: 0.7942 - val_loss: 0.5344 - val_accuracy: 0.7590
Epoch 4/10
782/782 [==============================] - 12s 16ms/step - loss: 0.4293 - accuracy: 0.8072 - val_loss: 0.4457 - val_accuracy: 0.7873
Epoch 5/10
782/782 [==============================] - 16s 21ms/step - loss: 0.3794 - accuracy: 0.8355 - val_loss: 0.4569 - val_accuracy: 0.7737
Epoch 6/10
782/782 [==============================] - 13s 17ms/step - loss: 0.3464 - accuracy: 0.8511 - val_loss: 0.4040 - val_accuracy: 0.8114
Epoch 7/10
782/782 [==============================] - 13s 16ms/step - loss: 0.3203 - accuracy: 0.8652 - val_loss: 0.4463 - val_accuracy:

In [21]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [22]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [23]:
rmsprop = keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [24]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 13s 15ms/step - loss: 0.6848 - accuracy: 0.5500 - val_loss: 0.6577 - val_accuracy: 0.6168
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5905 - accuracy: 0.6849 - val_loss: 0.5643 - val_accuracy: 0.7009
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5104 - accuracy: 0.7506 - val_loss: 0.5155 - val_accuracy: 0.7385
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4659 - accuracy: 0.7807 - val_loss: 0.5036 - val_accuracy: 0.7534
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4330 - accuracy: 0.8018 - val_loss: 0.4553 - val_accuracy: 0.7843
Epoch 6/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3992 - accuracy: 0.8243 - val_loss: 0.4300 - val_accuracy: 0.8003
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3753 - accuracy: 0.8388 - val_loss: 0.4095 - val_accuracy:

In [25]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3259 - accuracy: 0.8646 - val_loss: 0.3836 - val_accuracy: 0.8291
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3184 - accuracy: 0.8670 - val_loss: 0.3830 - val_accuracy: 0.8305
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3112 - accuracy: 0.8696 - val_loss: 0.3721 - val_accuracy: 0.8357
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3054 - accuracy: 0.8729 - val_loss: 0.3656 - val_accuracy: 0.8376
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3003 - accuracy: 0.8756 - val_loss: 0.3715 - val_accuracy: 0.8376
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.2952 - accuracy: 0.8762 - val_loss: 0.3668 - val_accuracy: 0.8351
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 0.2917 - accuracy: 0.8785 - val_loss: 0.3667 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation
